In [12]:
import numpy as np
import matplotlib.pyplot as plt
import sknetwork as skn
from scipy import sparse
from scipy.ndimage import gaussian_filter
from skimage.segmentation import watershed
from sknetwork.path import distance
from sknetwork.topology import get_connected_components

In [ ]:
#get image and define the image space
g = #image_here
image_range = zip(range(image.shape[0]),range(image.shape[1]))

In [2]:
def B_delta(p,delta): #rename suggestion: delta ball
    B = set()
    for q in image_range:
        if np.linalg.norm(p-q)<= delta:
            B.add(q)
    return B

### Algorithm I

In [ ]:
def generate_fragments(img,std_dev,int_treshold,min_seed_dist,max_search_depth,max_frag_dist):
    g = gaussian_filter(img,std_dev)
    g_copy = np.copy(g)
    
    #Fehlerabfangen
    
    if std_dev < 0:
        raise ValueError("smoothing strength needs to be positive")
    if int_threshold > 1 or int_threshold < 0:
        raise ValueError("relative intensity threshold needs to be between 0 and 1")
    if min_seed_dist < 1:
        raise ValueError("min seed distance needs to be >= 1")
    if max_search_depth < 0:
        raise ValueError("maximum search error needs to be positive")
    if max_frag_dist < 0:
        raise ValueError("maximum fragments distance needs to be positive")
        
    
    PI = []
    for p in image_range:
        B = B_delta(p,min_seed_dist)
        if (g[p]== max(g[q] for q in B)) and (1-int_treshold)*g[p]>=min(g[q] for q in B): 
            PI.append(p) 
            g_copy[p] = len(PI) 
        else:
            g_copy[p]=0   
    
    Omega = []
    Labels = watershed(img,markers=g_copy)
    for label in range(1,len(PI)+1):
        w = np.argwhere(Labels==label) #Das sind nur die indizies der Region
        Omega.append(w)
        
    Epsilon = np.zeros((len(Omega), len(Omega)))
    for i,_ in enumerate(Omega[:-1]):
        for j,_ in enumerate(Omega[i+1:]):
            #adjacency
            #local intensity peaks r(w)??
            if np.linalg.norm(PI[i] - PI[j]) <= max_frag_dist:
                Epsilon[i][j] = 1
                Epsilon[j][i] = 1
    Epsilon = sparse.csr_matrix(Epsilon) #needed to work with sknetwork pacakge
             
    
   # Epsilon = set()
   # for i,w in enumerate(Omega[:-1]):
   #     for j,v in enumerate(Omega[i+1:],start=i+1):
   #         #What is adjacency?
   #         if (x+(e,0) in v or x+(0,e) in v for x in w for e in (1,-1)) and np.linalg.norm(PI[i] - PI[j]) <= max_frag_dist:
   #             Epsilon.add({w,v})
            
    S = []
    #unsure what simply connected is supposed to indicate here
    for i in range(Omega):
        for #enter iteration of connected subgraphs:
            dis_matrix = distance(Epsilon, method = 'D') #return distance matrix
            if dis_matrix[i] <= h:
                #add set Z to S
    
    return Omega, S

###  testing out graphs

In [20]:
adjacency = np.array([[0, 1, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [0, 1, 0, 0]])
adjacency = sparse.csr_matrix(adjacency)

dist = distance(adjacency, method = 'D')
dist
dist[1].max()

1.0

### Algorithm II

In [ ]:
def global_solution(f,alpha,Omega,S):
    n = len(S) #stimmt n?
    u = np.zeros(n)
    V = Omega
    Z = S
    while V != ():
#work in progress
        c = np.empty(n)
        for k in range(len(S)):
            c[k] = (f[k] + alpha) / #abs value?
        k_min = argmin(c)
        u[k_min],V = 1, #v\{Z}?
    
    while Z!= set():
        k_prim = 